# Impulsivity

## Params to change


In [27]:
num_file = 1
data_source = "qualysis"
sliding_window_len = 80
rolling_mean_smoothing_window = 5
savgol_window_len = None
savgol_polyorder = None


feature_to_plot = 'Impulsivity'
signal = "trajectory"

In [28]:
import pandas as pd
from pathlib import Path
from io import StringIO

def read_tsv_qualysis(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    data_start_idx = next(i for i, line in enumerate(lines) if line.startswith("Frame"))
    header = lines[data_start_idx].strip().split('\t')
    data_lines = lines[data_start_idx + 1:]
    
    df = pd.read_csv(StringIO(''.join(data_lines)), sep='\t', names=header)
    return df

imp_dir = Path().cwd()
txt_files = sorted(imp_dir.glob("*.tsv"))
print(f"Reading file: {txt_files[num_file]}")
df = read_tsv_qualysis(txt_files[num_file])
df = df.iloc[:, 3:]
df.head()

Reading file: /home/gaggio/Documents/Research/pyeyesweb/PyEyesWeb/tests/benchmark/data/mid_level/trial0002_impulsive.tsv


,HEAD.BACK X,HEAD.BACK Y,HEAD.BACK Z,HEAD.FRONT.RIGHT X,HEAD.FRONT.RIGHT Y,HEAD.FRONT.RIGHT Z,HEAD.FRONT.LEFT X,HEAD.FRONT.LEFT Y,HEAD.FRONT.LEFT Z,SPINE_SHOULDER X,...,HAND_RIGHT Z,HAND_LEFT X,HAND_LEFT Y,HAND_LEFT Z,HAND_PINKIE_LEFT X,HAND_PINKIE_LEFT Y,HAND_PINKIE_LEFT Z,HAND_PINKIE_RIGHT X,HAND_PINKIE_RIGHT Y,HAND_PINKIE_RIGHT Z
0,-1280.662,-78.035,1392.360,-1073.467,10.229,1459.807,-1108.292,72.263,1462.900,-1302.062,...,771.558,-1234.651,243.070,757.166,-1251.740,274.897,707.227,-901.337,-136.499,710.622
1,-1274.524,-74.870,1391.382,-1067.589,13.350,1459.338,-1102.573,75.431,1462.274,-1295.975,...,770.959,-1228.428,246.026,755.856,-1245.578,277.822,705.986,-896.461,-132.093,709.755
2,-1268.478,-71.777,1390.353,-1061.701,16.509,1458.781,-1096.712,78.591,1461.644,-1289.971,...,769.979,-1222.420,248.978,754.318,-1239.796,280.604,704.423,-891.842,-127.853,708.406
3,-1262.377,-68.531,1389.301,-1055.839,19.438,1457.568,-1090.914,81.756,1461.026,-1283.910,...,768.669,-1216.763,251.773,752.516,-1234.322,283.304,702.634,-887.429,-123.960,707.010
4,-1256.341,-65.358,1388.294,-1049.866,22.940,1457.746,-1084.993,84.917,1460.360,-1277.294,...,767.082,-1211.283,254.440,750.531,-1229.151,285.935,700.649,-883.338,-120.336,705.127


In [29]:
import numpy as np
print("Before cleaning:")
if df.isna().any(axis=None) or (df == 0.0).any(axis=None):
    print("\033[91mThe dataframe contains NaN or zero values.\033[0m")
    
df.replace(0, np.nan, inplace=True)
df.interpolate(method='linear', inplace=True)
df.ffill(inplace=True)
df.bfill(inplace=True)  # Backfill to handle any remaining NaNs

print("After cleaning:")
if df.isna().any(axis=None) or (df == 0.0).any(axis=None):
    print("\033[91mWARNING: The dataframe contains NaN or zero values.\033[0m")
else:
    print("\033[92mThe dataframe does not contain NaN or zero values.\033[0m")

Before cleaning:
The dataframe contains NaN or zero values.
After cleaning:
The dataframe does not contain NaN or zero values.


In [30]:
# Extract marker position tables and sort columns by marker name
markersPosTableX = df.iloc[:,0::3]
markersPosTableY = df.iloc[:,1::3]
markersPosTableZ = df.iloc[:,2::3]

# Sort columns alphabetically by marker name
markersPosTableX = markersPosTableX.reindex(sorted(markersPosTableX.columns), axis=1)
markersPosTableY = markersPosTableY.reindex(sorted(markersPosTableY.columns), axis=1)
markersPosTableZ = markersPosTableZ.reindex(sorted(markersPosTableZ.columns), axis=1)

markersPosTableX = markersPosTableX.rename(columns=dict(zip(list(markersPosTableX),[name[:-2].strip() for name in list(markersPosTableX)])))
markersPosTableY = markersPosTableY.rename(columns=dict(zip(list(markersPosTableY),[name[:-2].strip() for name in list(markersPosTableY)])))
markersPosTableZ = markersPosTableZ.rename(columns=dict(zip(list(markersPosTableZ),[name[:-2].strip() for name in list(markersPosTableZ)])))

signalX = markersPosTableX.values
signalY = markersPosTableY.values
signalZ = markersPosTableZ.values
markersPosX = markersPosTableX.values
markersPosY = markersPosTableY.values
markersPosZ = markersPosTableZ.values

In [31]:
bones = [[0,4],[0,15],[5,1],[1,16],[15,13],[16,14],[14,19],[13,19],[19,20],[2,17],[3,18],[13,17],[14,18],[20,21],[21,17],[21,18],[17,23],[23,18],[22,23],[11,12],[10,12],[10,11],[10,21],[2,24],[6,24],[2,6],[6,7],[7,24],[3,9],[3,8],[8,25],[9,25],[9,8]]
markers_names = [col for col in markersPosTableX.columns]
bones_from_names = [(markers_names[bone[0]], markers_names[bone[1]]) for bone in bones]
print(bones_from_names, "\n", list(zip(markers_names, range(len(markers_names)))))

[('ANKLE_LEFT', 'FOOT_LEFT'), ('ANKLE_LEFT', 'KNEE_LEFT'), ('FOOT_RIGHT', 'ANKLE_RIGHT'), ('ANKLE_RIGHT', 'KNEE_RIGHT'), ('KNEE_LEFT', 'HIP_LEFT'), ('KNEE_RIGHT', 'HIP_RIGHT'), ('HIP_RIGHT', 'SPINE_BASE'), ('HIP_LEFT', 'SPINE_BASE'), ('SPINE_BASE', 'SPINE_MID'), ('ELBOW_LEFT', 'SHOULDER_LEFT'), ('ELBOW_RIGHT', 'SHOULDER_RIGHT'), ('HIP_LEFT', 'SHOULDER_LEFT'), ('HIP_RIGHT', 'SHOULDER_RIGHT'), ('SPINE_MID', 'SPINE_SHOULDER'), ('SPINE_SHOULDER', 'SHOULDER_LEFT'), ('SPINE_SHOULDER', 'SHOULDER_RIGHT'), ('SHOULDER_LEFT', 'STERNUM_MID'), ('STERNUM_MID', 'SHOULDER_RIGHT'), ('STERNUM_BASE', 'STERNUM_MID'), ('HEAD.FRONT.LEFT', 'HEAD.FRONT.RIGHT'), ('HEAD.BACK', 'HEAD.FRONT.RIGHT'), ('HEAD.BACK', 'HEAD.FRONT.LEFT'), ('HEAD.BACK', 'SPINE_SHOULDER'), ('ELBOW_LEFT', 'WRIST_LEFT'), ('HAND_LEFT', 'WRIST_LEFT'), ('ELBOW_LEFT', 'HAND_LEFT'), ('HAND_LEFT', 'HAND_PINKIE_LEFT'), ('HAND_PINKIE_LEFT', 'WRIST_LEFT'), ('ELBOW_RIGHT', 'HAND_RIGHT'), ('ELBOW_RIGHT', 'HAND_PINKIE_RIGHT'), ('HAND_PINKIE_RIGHT', 'W

In [32]:
print(markersPosTableX.iloc[0, markers_names.index("HAND_LEFT")], markersPosTableY.iloc[0, markers_names.index("HAND_LEFT")], markersPosTableZ.iloc[0, markers_names.index("HAND_LEFT")])

-1234.651 243.07 757.166


In [33]:
import numpy as np

def get_adjacency_matrix(numMarkers, bones):
    edges = np.array(bones)
    adjacencyMatrix = np.zeros((numMarkers,numMarkers),dtype=bool)
    adjacencyMatrix[edges[:,0],edges[:,1]] = True
    return adjacencyMatrix

adjacencyMatrix = get_adjacency_matrix(len(markersPosTableX.columns),bones)

selected_joints = ["HAND_RIGHT", "HAND_LEFT"]

In [34]:
import tqdm
from pyeyesweb.mid_level.impulsivity import ImpulsivityNiewiadomski
from pyeyesweb.data_models.sliding_window import SlidingWindow

# Calculate the bilateral symmetry of the two hands using the BilateralSymmetry
sw = SlidingWindow(max_length=sliding_window_len, n_dimensions=3, m_joints=1)
imp = ImpulsivityNiewiadomski()
values = {joint : {"suddenness": np.zeros(signalX.shape[0],np.float64), "direction_change": np.zeros(signalX.shape[0],np.float64), "value": np.zeros(signalX.shape[0],np.float64)} for joint in selected_joints}
for joint in selected_joints:
    joint_idx = markers_names.index(joint)
    res = imp(np.array([signalX[:, joint_idx], signalY[:, joint_idx], signalZ[:, joint_idx]]).T)
    for frame in tqdm.tqdm(range(signalX.shape[0])):
        values[joint]["suddenness"][frame] = res["suddenness"][frame]
        values[joint]["direction_change"][frame] = res["direction_change"][frame]
        values[joint]["value"][frame] = res["value"][frame]
        
print(signalX[0, markers_names.index("HAND_LEFT")], signalY[0, markers_names.index("HAND_LEFT")], signalZ[0, markers_names.index("HAND_LEFT")])

100%|██████████| 5813/5813 [00:00<00:00, 1402363.35it/s]

-1234.651 243.07 757.166


In [35]:
for joint in values:
    values[joint]["suddenness"] = np.nan_to_num(values[joint]["suddenness"])
    values[joint]["direction_change"] = np.nan_to_num(values[joint]["direction_change"])
    values[joint]["value"] = np.nan_to_num(values[joint]["value"])
joint_name = "HAND_RIGHT"
values_to_plot = [values[joint_name]["value"], values[joint_name]["suddenness"], values[joint_name]["direction_change"]]
colors_to_plot = ["k", "k", "k"]

In [36]:
import json

with open("acc_annotations.json", "r") as f:
    annotation = json.load(f)[txt_files[num_file].name]
    annotation = [anno for anno in annotation if anno < signalX.shape[0]]

In [38]:
%matplotlib qt

from matplotlib import pyplot as plt
from matplotlib.widgets import Slider
import numpy as np
from matplotlib.widgets import Button
import matplotlib.animation as animation
#bones = infer_bones(df)
import os

import json
import time
import ast

#bones = infer_bones(df)
#print(bones)

if 'anim' in globals() and anim.event_source != None:
    anim.pause()



# Convert bones into adjacency matrix
bones_file_path = imp_dir / "bones_from_names.txt"
bones_names = []
if bones_file_path.exists():
    with open(bones_file_path, "r") as f:
        bones_names = [ast.literal_eval(l.strip()) for l in f.readlines() if l.strip()]

bones = []
for b in bones_names:
    if b[0] in markers_names and b[1] in markers_names:
        bones.append([markers_names.index(b[0]), markers_names.index(b[1])])


fig = plt.figure(f"3D Movement file:{num_file} joint:{joint_name} signal:{signal}", figsize=(19.20, 10.80))
plt.clf()

# Add a 3D subplot with the specified position and size
ax = fig.add_axes([0, -0.16, 1, .85], projection='3d')
ax.view_init(elev=0, azim=0)
# Create three stacked subplots for the synchronization plots
left, bottom, width, total_height = 0.05, 0.55, 0.9, 0.42
gap = 0.04
height = (total_height - 2 * gap) / 3

ax_acc1 = fig.add_axes([left, bottom + 2 * (height + gap), width, height])
ax_acc2 = fig.add_axes([left, bottom + (height + gap), width, height])
ax_acc3 = fig.add_axes([left, bottom, width, height])


ax_acc1.plot(values_to_plot[0], color="blue", linewidth=1, label="Impulsivity")
ax_acc1.scatter(annotation, values_to_plot[0][annotation]*0+1.1, marker=".", color='red', linewidth=2, label="Annotation")
for anno in annotation:
    ax_acc1.axvline(x=anno, color='red', linestyle='--', linewidth=0.5)
ax_acc2.plot(values_to_plot[1], color="blue", linewidth=1, label="Suddenness")
ax_acc3.plot(values_to_plot[2], color="blue", linewidth=1, label="Direction Change")

ax_acc1.legend(loc='best')
ax_acc2.legend(loc='best')
ax_acc3.legend(loc='best')
ax_acc1.set_yticks([0, 0.7, 1])
ax_acc2.set_yticks([0, 0.7, 1])
ax_acc3.set_yticks([0, 0.7, 1])
ax_acc1.set_xticks(list(range(0, len(values_to_plot[2]), 300)))
ax_acc1.set_xticklabels([])  # Hide x-axis ticks for the first two subplots
ax_acc2.set_xticks(list(range(0, len(values_to_plot[2]), 300)))
ax_acc2.set_xticklabels([])  # Hide x-axis ticks for the first two subplots
ax_acc3.set_xticks(list(range(0, len(values_to_plot[2]), 300)))  # Show x-axis ticks for the last subplot
ax_acc3.set_ylim(-0.05, 1.05)
ax_acc1.set_ylim(-0.05, 1.2)
ax_acc2.set_ylim(-0.05, 1.05)
ax_acc3.set_ylim(-0.05, 1.05)
#ax_acc.plot(speed, color='green')
#ax_acc.set_yticks(np.linspace(0,max(acceleration),10).tolist()+ [max(acceleration)+0.05, max(acceleration)+0.55])
#ax_acc.set_yticklabels(np.round(np.linspace(0,max(acceleration),10),3).tolist()+ ["v = 0", f"v = {max(speed):.2f}"])
ax_acc1.set_title(f'Feature: {feature_to_plot} | Signal: {signal} | Joint: {joint_name} | Sliding window len = {sliding_window_len}', fontsize=10)
ax_acc3.set_xlabel('F r a m e')
#ax_acc.grid(axis='y', color='gray')
#ax_acc1.set_ylabel('Impulsivity')
ax_acc2.set_ylabel('Score')
#ax_acc3.set_ylabel('Direction Change')

#ax_acc.plot(peaks, acceleration[peaks], 'ro', markersize=5, label='Peaks')
ax_acc1.grid(axis='both', color="#cccccc")
ax_acc2.grid(axis='both', color="#cccccc")
ax_acc3.grid(axis='both', color="#cccccc")

cursor1 = ax_acc1.plot(0, values_to_plot[0][0], 'ko', markersize=4, label='Current Frame')
cursor2 = ax_acc2.plot(0, values_to_plot[1][0], 'ko', markersize=4, label='Current Frame')
cursor3 = ax_acc3.plot(0, values_to_plot[2][0], 'ko', markersize=4, label='Current Frame')
cursor = [cursor1, cursor2, cursor3]

# Calculate movement boundaries for the view
minMax = np.zeros((2,3))
minMax[0,0] = np.nanmin(markersPosX)
minMax[0,1] = np.nanmin(markersPosY)
minMax[0,2] = np.nanmin(markersPosZ)
minMax[1,0] = np.nanmax(markersPosX)
minMax[1,1] = np.nanmax(markersPosY)
minMax[1,2] = np.nanmax(markersPosZ)
#minMax = np.array([[-2292.197, -2213.251, 15.017],[ 617.451, 760.748, 1815.185]])


# Set appropriate axis limits
ax.set_xlim([minMax[0,0],minMax[1,0]])
ax.set_ylim([minMax[0,1],minMax[1,1]])
ax.set_zlim([minMax[0,2],minMax[1,2]])

# Set the window title
#fig.canvas.manager.window.title("3D Movement")


# Set background white and grid lines white to mimic transparency
plt.rcParams['grid.color'] = 'white'
ax.tick_params(axis='both', colors='white')

# Remove box faces except the ground
ax.xaxis.pane.fill = False
ax.yaxis.pane.fill = False
ax.zaxis.pane.fill = True
#ax.w_zaxis.pane.set_facecolor("w")
ax.xaxis.pane.set_edgecolor("w")
ax.yaxis.pane.set_edgecolor('w')
ax.zaxis.pane.set_edgecolor("w")

# Hide axis lines and ticks
ax.xaxis.line.set_visible(False)
ax.yaxis.line.set_visible(False)
ax.zaxis.line.set_visible(False) 


def update_plot(val):
    #for annotation in ax.texts:
    #    annotation.remove()

    # filter the data based on the current time index
    filteredX = markersPosX[val]
    filteredY = markersPosY[val]
    filteredZ = markersPosZ[val]
    
    # Update scatter plot data instead of creating a new scatter plot
    scatter_plot = ax.collections[0]
    #scatter_plot.set_offsets(np.column_stack((filteredX, filteredY)))
    scatter_plot._offsets3d = (filteredX, filteredY, filteredZ)

    # Update bones based on the adjacency matrix
    k = 0
    for i in range(len(adjacencyMatrix)):
        for j in range(0, len(adjacencyMatrix)):
            if adjacencyMatrix[i, j]:

                # Find the existing line corresponding to this edge and update its coordinates
                line = allLines[k]
                line.set_xdata([filteredX[i], filteredX[j]])
                line.set_ydata([filteredY[i], filteredY[j]])
                line.set_3d_properties([filteredZ[i], filteredZ[j]])
                k+=1
    
    # Add labels for each marker
    #for idx, (x, y, z) in enumerate(zip(filteredX, filteredY, filteredZ)):
    #    ax.text(x, y, z, markers_names[idx], color='black', fontsize=6)

    for id, cursor in enumerate([cursor1, cursor2, cursor3]):
        cursor[0].set_xdata([val])
        cursor[0].set_ydata([values_to_plot[id][val]])
    
    fig.canvas.draw_idle()


filteredX = markersPosX[0]
filteredY = markersPosY[0]
filteredZ = markersPosZ[0]

allLines = []

# Plot the first scatters
#print(markersPosX[0].shape, markersPosY[0].shape, markersPosZ[0].shape)
colors = ["red"] * markersPosX.shape[1]

colors[markers_names.index(joint_name)] = "blue"
print(markers_names.index(joint_name))
ax.scatter(markersPosX[0],markersPosY[0],markersPosZ[0], s=20, c=colors)
for i in range(len(adjacencyMatrix)):
    for j in range(len(adjacencyMatrix)):
        if adjacencyMatrix[i,j]:
            ax.plot([filteredX[i], filteredX[j]],
                    [filteredY[i], filteredY[j]],
                    [filteredZ[i], filteredZ[j]],
                    color='k', linestyle='-', linewidth=0.7)
            allLines.append(ax.lines[-1])


# Create a slider widget
slider_ax = plt.axes([0.08, 0, 0.8, 0.03])
maxValue = len(markersPosX)-1
slider = Slider(slider_ax, 'Frame:', 0, maxValue, valinit=0, valstep=1)

# Register the update_plot function with the slider widget
slider.on_changed(update_plot)

# Function to update the plot for animation
def animate(frame):
    slider.set_val(frame)

step = 1
anim = animation.FuncAnimation(fig, slider.set_val, frames=range(0, maxValue, step), interval=1, repeat=True, cache_frame_data=False)

last_ = (time.perf_counter(), 0)
remaining_time = [1]

def _print_progress(current_frame, total_frames):
    global last_
    now = time.perf_counter()
    dt = now - last_[0]
    last_frame = last_[1]
    last_ = (now, current_frame)
    if last_frame % 5 == 2:
        remaining_time.append(dt * (total_frames - current_frame))
        if len(remaining_time) > 100:
            remaining_time.pop(0)
        avg_remaining_time = sum(remaining_time) / len(remaining_time)
        print(f"Saving frame {current_frame}/{total_frames} | remaining_time={avg_remaining_time:.4f}s", end="\r")

anim.save(imp_dir / f"result_{txt_files[num_file].stem}_{feature_to_plot}.mp4", writer='ffmpeg', fps=100, progress_callback=_print_progress)


# Create a button to pause and play the animation
button_ax = plt.axes([0.92, 0.005, 0.06, 0.03])
button = Button(button_ax, 'Pause/Play')
running = True

# Function to pause and play the animation
def pause_play(event):
    global running 
    if running:
        anim.pause()
    else: 
        anim.resume()
    running ^= True

button.on_clicked(pause_play)
anim.pause()

# Show the plot
plt.show()

9
